In [1]:
import os
import pymysql
import pyrootutils
import pandas as pd

pyrootutils.setup_root(os.curdir, indicator=".project-root", pythonpath=True)
from Lambda.product_db_range import rds_info
import json
import boto3

conn = pymysql.connect(
    host=rds_info.db_host,
    user=rds_info.db_username,
    passwd=rds_info.db_password,
    db=rds_info.db_name,
    port=rds_info.db_port,
)

In [9]:
cursor = conn.cursor()
query = """
SELECT * FROM PRODUCT WHERE PRODUCT_ID BETWEEN 1 AND 5;
"""
cursor.execute(query)
products = cursor.fetchall()

In [10]:
product = products[0]
product

(1,
 88200,
 '다이브 하프 셔츠 CLEAR BLACK',
 'M',
 2,
 7,
 'https://lmood.co.kr/product/detail.html?product_no=2867&cate_no=198&display_group=1',
 1,
 '2023-08-13 00:24:27.txt')

In [14]:
query = f"""
SELECT * FROM MALL WHERE MALL_ID={product[7]};
"""
cursor.execute(query)
mall = cursor.fetchall()

In [16]:
mall[0]

(1,
 '엘무드',
 'https://lmood.co.kr/index.html',
 '엘무드(LMOOD)는 미니멀과 컴포트를 중점으로 이 시대 트렌드를 반영한 컨템포러리 감성 브랜드입니다.',
 '2023-08-13 00:24:23.jpg')

In [19]:
if product[4] == 2: # top
    query = f"""
    SELECT * FROM SIZE 
    JOIN TOP_SIZE ON SIZE.TOP_ID = TOP_SIZE.TOP_ID
    WHERE PRODUCT_ID = {product[0]};
    """
    cursor.execute(query)
sizes = cursor.fetchall()
sizes

((1, '44', 1, 1, None, None, None, 1, 75.0, 53.0, 60.5, 24.5),
 (2, '46', 1, 2, None, None, None, 2, 76.0, 54.5, 63.0, 25.5),
 (3, '48', 1, 3, None, None, None, 3, 77.0, 56.0, 65.5, 26.5))

In [11]:
query = f"""
SELECT * FROM IMAGEPATH
WHERE PRODUCT_ID = {product[0]}
"""
cursor.execute(query)
image_paths = cursor.fetchall()
image_paths[]

((1, '2023-08-13 00:24:27.jpg', 1, 1),
 (2, '2023-08-13 00:24:27.jpg', 1, 0),
 (3, '2023-08-13 00:24:28.jpg', 1, 0),
 (4, '2023-08-13 00:24:28.jpg', 1, 0),
 (5, '2023-08-13 00:24:29.jpg', 1, 0),
 (6, '2023-08-13 00:24:29.jpg', 1, 0),
 (7, '2023-08-13 00:24:30.jpg', 1, 0),
 (8, '2023-08-13 00:24:30.jpg', 1, 0),
 (9, '2023-08-13 00:24:30.jpg', 1, 0),
 (10, '2023-08-13 00:24:31.jpg', 1, 0),
 (11, '2023-08-13 00:24:31.jpg', 1, 0))

In [12]:
def handler(event, context):
    product_range = event['range']
    cursor = conn.cursor()
    query = f"""
    SELECT * FROM PRODUCT WHERE PRODUCT_ID BETWEEN {product_range[0]} AND {product_range[1]};
    """
    cursor.execute(query)
    products = cursor.fetchall()

    body = []
    for product in products:
        row = {}
        row['product'] = {key: value for key, value in zip(rds_info.product_keys, product)}

        query = f"""
        SELECT * FROM MALL WHERE MALL_ID={row['product']['mall_id']};
        """
        cursor.execute(query)
        mall = cursor.fetchall()[0]
        row['mall'] = {key: value for key, value in zip(rds_info.mall_keys, mall)}

        if row['product']['category_id'] == 1:
            query = f"""
            SELECT * FROM SIZE 
            JOIN OUTER_SIZE ON SIZE.OUTER_ID = OUTER_SIZE.OUTER_ID
            WHERE PRODUCT_ID = {row['product']['product_id']};
            """
            cursor.execute(query)
            sizes = cursor.fetchall()
            
            row['size'] = []
            for size in sizes:
                size_dict = {key: value for key, value in zip(rds_info.outer_keys, size[-4:])}
                size_dict['name'] = size[1]
                row['size'] += [size_dict]
            
        
        if row['product']['category_id'] == 2:
            query = f"""
            SELECT * FROM SIZE 
            JOIN TOP_SIZE ON SIZE.TOP_ID = TOP_SIZE.TOP_ID
            WHERE PRODUCT_ID = {row['product']['product_id']};
            """
            cursor.execute(query)
            sizes = cursor.fetchall()
            
            row['size'] = []
            for size in sizes:
                size_dict = {key: value for key, value in zip(rds_info.top_keys, size[-4:])}
                size_dict['name'] = size[1]
                row['size'] += [size_dict]
        

        if row['product']['category_id'] == 3:
            query = f"""
            SELECT * FROM SIZE 
            JOIN DRESS_SIZE ON SIZE.DRESS_ID = DRESS_SIZE.DRESS_ID
            WHERE PRODUCT_ID = {row['product']['product_id']};
            """
            cursor.execute(query)
            sizes = cursor.fetchall()
            
            row['size'] = []
            for size in sizes:
                size_dict = {key: value for key, value in zip(rds_info.dress_keys, size[-9:])}
                size_dict['name'] = size[1]
                row['size'] += [size_dict]

        
        if row['product']['category_id'] == 3:
            query = f"""
            SELECT * FROM SIZE 
            JOIN BOTTOM_SIZE ON SIZE.BOTTOM_ID = BOTTOM_SIZE.BOTTOM_ID
            WHERE PRODUCT_ID = {row['product']['product_id']};
            """
            cursor.execute(query)
            sizes = cursor.fetchall()
            
            row['size'] = []
            for size in sizes:
                size_dict = {key: value for key, value in zip(rds_info.bottom_keys, size[-6:])}
                size_dict['name'] = size[1]
                row['size'] += [size_dict]

        
        query = f"""
        SELECT * FROM IMAGEPATH
        WHERE PRODUCT_ID = {row['product']['product_id']} AND THUMBNAIL = 1;
        """
        cursor.execute(query)

        row['imagepath'] = []
        row['imagepath'] += [cursor.fetchall()[0][1]]
        
        query = f"""
        SELECT * FROM IMAGEPATH
        WHERE PRODUCT_ID = {row['product']['product_id']} AND NOT THUMBNAIL = 1;
        """
        cursor.execute(query)
        row['imagepath'] += [imagepath[1] for imagepath in cursor.fetchall()]

        body += [row]

    return body

In [20]:
event = {'range': (1, 5)}
response = json.dumps(handler(event, None)[0])

In [15]:
json.loads()

In [17]:
print(a)

엘무드(LMOOD)는 미니멀과 컴포트를 중점으로 이 시대 트렌드를 반영한 컨템포러리 감성 브랜드입니다.
